Copyright 2024 - Forusone : shins777@gmail.com

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

   https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

# Gemini Pro - Using Google Search as a grounding service.


# Configuration
## Install python packages
* Vertex AI SDK for Python
  * https://cloud.google.com/python/docs/reference/aiplatform/latest


In [ ]:
%pip install --upgrade --quiet google-cloud-aiplatform

In [ ]:
from IPython.display import display, Markdown

## Authentication to access to the GCP & Google drive

* Use OAuth to access the GCP environment.
 * Refer to the authentication methods in GCP : https://cloud.google.com/docs/authentication?hl=ko

In [ ]:
#  For only colab to authenticate to get an access to the GCP.
import sys

if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()

    # To access contents in Google drive
    # from google.colab import drive
    # drive.mount('/content/drive')

# Execute the example
## Set the environment on GCP Project
* Configure project information
  * Model name : LLM model name : https://cloud.google.com/vertex-ai/generative-ai/docs/learn/models
  * Project Id : prodect id in GCP
  * Region : region name in GCP

In [ ]:
MODEL_NAME="gemini-1.5-flash"
PROJECT_ID="ai-hangsik"
REGION="asia-northeast3"

### Vertex AI initialization
Configure Vertex AI and access to the foundation model.
* Vertex AI initialization : aiplatform.init(..)
  * https://cloud.google.com/python/docs/reference/aiplatform/latest#initialization

In [ ]:
import vertexai
from vertexai.generative_models import GenerativeModel, Part, Tool
import vertexai.generative_models as generative_models

# Initalizate the current vertex AI execution environment.
vertexai.init(project=PROJECT_ID, location=REGION)

# Access to the generative model.
model = GenerativeModel(MODEL_NAME)

## Prompt sent to LLM

In [ ]:
PROMPT = (
    "What is the latest news related to the key interest rate announced by FED after mid of May 2024"
)

## Response without grounding services.
This response is information from LLM without grounding services to enrich contexts sent to LLM.

In [ ]:
response = model.generate_content(PROMPT)
display(Markdown(response.text))

##Configuration Google Search

In [ ]:
from vertexai.generative_models import GenerationConfig
# Grounding service is still in preview.
from vertexai.preview.generative_models import grounding

def get_response(prompt:str)->str:

  tool = Tool.from_google_search_retrieval(grounding.GoogleSearchRetrieval())

  response = model.generate_content(
      prompt,
      tools=[tool],
      generation_config=GenerationConfig(
          temperature=0.0,
      ),
  )

  return response.text




In [ ]:
outcome = get_response(PROMPT)
display(Markdown(outcome))


## Fact finding

In [ ]:
from IPython.display import Image

image_path = "/content/drive/MyDrive/projects/google_gen_ai_sample/contents/images/"
image_file = "grounding_service_google_search.png"
Image(f"{image_path}{image_file}", width=700, height=300)
